In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing, SEIRHD_Severity

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from utils.create_report import create_report
from utils.loss import Loss_Calculator
from viz import plot_forecast

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Get Variable Params

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.5),
    'T_inc': (4, 5),
    'T_inf': (3, 4),
    'T_recov_mild': (20, 30),
    'T_recov_moderate': (20, 30),
    'T_recov_severe': (44, 45),
    'P_moderate': (0, 0.5),
    'P_severe': (0, 0.1),
    'P_fatal': (0, 0.1),
    'E_hosp_ratio': (0, 2),
    'I_hosp_ratio': (0, 1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, data_from_tracker=True, granular_data=True, filename=None, #Data
    model=SEIRHD_Severity, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
    train_period=14, val_period=0, num_evals=1500, initialisation='intermediate', #Optimisation related parameters
    which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased', 'stable_asymptomatic', 'stable_symptomatic', 'critical'], #Compartments to Apply Loss on 
    smooth_jump=True) #Smoothing

# predictions_dict['m2'] = single_fitting_cycle(
#     state, district, train_period=7, val_period=0, num_evals=700,
#     data_from_tracker=True, initialisation='intermediate', model=SEIRHD, 
#     which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'],
#     smooth_jump=False)
    
predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m1']['data_last_date']

In [ ]:
predictions_dict['m1']['best_params']

In [ ]:
predictions_dict['m1']['df_prediction'].iloc[:7, :].loc[:, ['date', 'critical']]

In [ ]:
predictions_dict['m1']['df_train'].iloc[-7:, :].loc[:, ['date', 'critical']]

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss DataFrame

In [ ]:
predictions_dict['m2']['df_loss']

## Plot Forecasts